# Stochastic Grid Search for the optimization of multiple hyperparameters of random forest for regression

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
housing = pd.read_csv("housing.csv")
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [3]:
housing.shape

(20640, 10)

### Data pre-processing: remove NaN

In [4]:
housing.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [5]:
# there are 207 NaN in the column 'total_bedrooms', remove them
# check housing again using housing.isnull().sum()
housing['total_bedrooms'].fillna((housing['total_bedrooms'].mean()), inplace=True)
housing.isnull().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

### Data pre-processing: Convert categorical data to numerical data - "ocean_proximity"

In [6]:
housing["ocean_proximity"].value_counts()

<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64

In [7]:
#We can use one-hot encoding method by calling pd.get_dummies
categorical_columns=['ocean_proximity'] # must be a list
housing = pd.get_dummies(housing, columns=categorical_columns)
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,0,0,0,1,0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,0,0,0,1,0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,0,0,0,1,0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,0,0,0,1,0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,0,0,0,1,0


### Visualize the data

In [8]:
from skimage.io import imread
california_img = imread('california.png')
x1 = min(housing["longitude"].values)
x2 = max(housing["longitude"].values)
y1 = min(housing["latitude"].values)
y2 = max(housing["latitude"].values)
#---------------------------------------------
# the parameter c refers to color
# thus, median_house_value is color-coded in the left plot
fig, ax =plt.subplots(1,2)
housing.plot(ax=ax[0], kind="scatter", x="longitude", y="latitude",
             s=housing['population']/100, label="population",
             c="median_house_value", cmap=plt.get_cmap("jet"),
             colorbar=True, alpha=0.4, figsize=(10,7))
#---------------------------------------------
# the parameter c refers to color
# thus, median_income is color-coded in the right plot
ax[0].imshow(california_img,extent=[x1,x2,y1,y2])
ax[0].set_title('median_house_value')
housing.plot(ax=ax[1], kind="scatter", x="longitude", y="latitude",
             s=housing['population']/100, label="population",
             c="median_income", cmap=plt.get_cmap("jet"),
             colorbar=True, alpha=0.4, figsize=(10,7))
ax[1].imshow(california_img,extent=[x1,x2,y1,y2])
ax[1].set_title('median_income')

ImportError: cannot import name '_validate_lengths' from 'numpy.lib.arraypad' (/Users/zhukson/anaconda3/lib/python3.7/site-packages/numpy/lib/arraypad.py)

### Prepare the Training, Validation and Testing Datasets

In [9]:
X=housing.drop(['median_house_value'], axis=1)
X.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,0,0,0,1,0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,0,0,0,1,0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,0,0,0,1,0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,0,0,0,1,0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,0,0,0,1,0


In [10]:
Y=housing['median_house_value']
Y.head()

0    452600.0
1    358500.0
2    352100.0
3    341300.0
4    342200.0
Name: median_house_value, dtype: float64

In [11]:
# convert pandas dataframe/series to numpy array
# sklearn functions may not work well with pandas data types
X_columns=X.columns #store the column names
X=X.values
Y=Y.values

In [12]:
#trainnig, validation, testing split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
#split X_train and Y_train into a 'pure' training set and a validation set
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=0)
print('train:', X_train.shape, Y_train.shape)
print('validation:', X_val.shape, Y_val.shape)
print('test:', X_test.shape, Y_test.shape)

train: (14860, 13) (14860,)
validation: (1652, 13) (1652,)
test: (4128, 13) (4128,)


In [13]:
#apply feature normalization to training, validation and test sets
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(X_train) # think about why fit to X_train, not X ?
X_train=scaler.transform(X_train)
X_val=scaler.transform(X_val)
X_test=scaler.transform(X_test)

In [14]:
type(X_train)

numpy.ndarray

In [15]:
Y_test.shape

(4128,)

### APPLY Stochastic Grid Search TO OPTIMIZE HYPERPARAMETER

In [16]:
import numpy as np
n_estimators = [20,100,200]
max_depth = np.arange(1,101,11)
max_features = ["auto", "sqrt"]
min_samples_split = [2,5,10]
min_samples_leaf = [1,2,4]
max_samples=[0.5,0.8,1]

In [17]:
param = {
    "n_estimators": n_estimators,
    "max_features": max_features,
    "max_depth": max_depth,
    "min_samples_split": min_samples_split,
    "min_samples_leaf": min_samples_leaf,
    "max_samples": max_samples
}

### randomly select samples from the grid.

In [18]:
from sklearn.ensemble import RandomForestRegressor
#the implementation would be:
#for loop(total times of sampling from the grid):
#    randomly sample from the grid
#    build a new model based on this random sample
#    calculate the MAE lost on training set and validation set.
#    compare the MAE validation lost with the best MAE validation lost.
#    If it is smaller than it, update the best MAE validation lost and the best hyper parameters.
#    Otherwise, continue loop.
import random

MAE_train_list=[]
MAE_val_list=[]
bestscore=100000
bestparam=[]

i_iterators = 100 #selection times
for i in range(i_iterators): 
    random.seed(i) #use seed to make it repeatable
    sample_param = {x: random.sample(list(y), 1)[0] for x, y in param.items()}  #randomly sample from the grid
    estimators = sample_param['n_estimators'] 
    m_depth = sample_param['max_depth']
    m_features = sample_param['max_features']
    mi_samples_split = sample_param['min_samples_split']
    mi_samples_leaf = sample_param['min_samples_leaf']
    ma_samples = sample_param['max_samples']
    #build a model based on these ramdonly selected parameters
    rf = RandomForestRegressor(n_estimators=estimators,random_state=0,
                               max_depth=m_depth,max_features=m_features, 
                               min_samples_split=mi_samples_split, 
                               min_samples_leaf=mi_samples_leaf,max_samples=ma_samples,n_jobs = -1)
    rf.fit(X_train,Y_train)
    #calculate the MAE error on both training set and validation set and put them into MAE_train_list and MAE_val_list.
    y_train_pred = rf.predict(X_train)
    MAE_training=np.mean(np.abs(y_train_pred-Y_train))
    MAE_train_list.append(MAE_training)
    y_val_pred = rf.predict(X_val)
    MAE_val = np.mean(np.abs(y_val_pred-Y_val))
    MAE_val_list.append(MAE_val)
    # if current model's MAE_val error is smaller than the bestscore, update the bestscore and the best parameter.
    if MAE_val<bestscore:
        bestscore=MAE_val
        bestparam=sample_param

In [23]:
print(bestparam,bestscore)

{'n_estimators': 100, 'max_features': 'auto', 'max_depth': 89, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_samples': 0.8} 32991.53932448073


In [24]:
estimators = bestparam['n_estimators'] 
m_depth = bestparam['max_depth']
m_features = bestparam['max_features']
mi_samples_split = bestparam['min_samples_split']
mi_samples_leaf = bestparam['min_samples_leaf']
ma_samples = bestparam['max_samples']
rf = RandomForestRegressor(n_estimators=estimators,random_state=0,
                               max_depth=m_depth,max_features=m_features, 
                               min_samples_split=mi_samples_split, 
                               min_samples_leaf=mi_samples_leaf,max_samples=ma_samples)
rf.fit(X_train,Y_train)
Y_test_pred=rf.predict(X_test)
MSE = np.mean((Y_test - Y_test_pred)**2)
MAE = np.mean(np.abs(Y_test - Y_test_pred))
MAPE =  np.mean(np.abs(Y_test - Y_test_pred)/Y_test)
print('MSE=', MSE)
print('MAE=', MAE)
print('MAPE=', MAPE)

MSE= 2361986186.596001
MAE= 31926.309208299514
MAPE= 0.18052419484706364


### now compare this method with RandomizedSearchCV

In [25]:
from sklearn.model_selection import RandomizedSearchCV
randomcv = RandomizedSearchCV(RandomForestRegressor(), param, n_iter=100, cv=3,n_jobs = -1)
randomcv.fit(X_train, Y_train)

RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'max_depth': array([  1,  12,  23,  34,  45,  56,  67,  78,  89, 100]),
                                        'max_features': ['auto', 'sqrt'],
                                        'max_samples': [0.5, 0.8, 1],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [20, 100, 200]})

In [27]:
randomcv.best_params_

{'n_estimators': 200,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_samples': 0.8,
 'max_features': 'auto',
 'max_depth': 67}

In [28]:
estimators = randomcv.best_params_['n_estimators'] 
m_depth = randomcv.best_params_['max_depth']
m_features = randomcv.best_params_['max_features']
mi_samples_split = randomcv.best_params_['min_samples_split']
mi_samples_leaf = randomcv.best_params_['min_samples_leaf']
ma_samples = randomcv.best_params_['max_samples']
rf = RandomForestRegressor(n_estimators=estimators,random_state=0,
                               max_depth=m_depth,max_features=m_features, 
                               min_samples_split=mi_samples_split, 
                               min_samples_leaf=mi_samples_leaf,max_samples=ma_samples)
rf.fit(X_train,Y_train)
Y_test_pred=rf.predict(X_test)
MSE = np.mean((Y_test - Y_test_pred)**2)
MAE = np.mean(np.abs(Y_test - Y_test_pred))
MAPE =  np.mean(np.abs(Y_test - Y_test_pred)/Y_test)
print('MSE=', MSE)
print('MAE=', MAE)
print('MAPE=', MAPE)

MSE= 2329153076.075619
MAE= 31806.578974079464
MAPE= 0.18031939271392192


#### clearly, we can see the MAPE error between randomizedSearchCV and my method is almost the same.